## Problem 1: Generating Random Boolean Functions

### Problem Description

The [Deutsch–Jozsa algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa) works with functions that accept a fixed number of [Boolean inputs](https://realpython.com/python-boolean/) and return a single Boolean output.
Each function is guaranteed to be either **constant** (always returns `False` or always returns `True`) or **balanced** (returns `True` for exactly half of the possible input combinations).

The task is to write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of constant or balanced functions taking **four** Boolean arguments as inputs.

### My Understanding

We need to build a function that, each time it is called, produces and returns a new **callable function** `f(b1, b2, b3, b4)` → `bool`.
The returned function must behave as a [black box](https://en.wikipedia.org/wiki/Black_box) — a caller can evaluate it on inputs but should not easily see its internal logic.
This is important because later problems will test classical and quantum strategies for determining whether the function is constant or balanced, and those strategies must rely on calling `f` rather than inspecting it.

### Theory / Background

With four Boolean inputs, there are $2^4 = 16$ possible input combinations ([Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product) of $\{\text{False}, \text{True}\}$ taken four times).
A Boolean function $f$ assigns an output to each of these, and this mapping is its [truth table](https://en.wikipedia.org/wiki/Truth_table).

There are exactly **two constant** functions (all outputs `False`, or all outputs `True`) and $\binom{16}{8} = 12{,}870$ **balanced** functions (choosing which 8 of the 16 inputs map to `True`, as described by the [binomial coefficient](https://en.wikipedia.org/wiki/Binomial_coefficient)).
Together these give 12,872 valid functions — a small subset of the $2^{16} = 65{,}536$ total Boolean functions of four inputs.

### Approach

Following the truth-table-with-closure pattern from the module materials ([Python Closures: Common Use Cases and Examples](https://realpython.com/python-closure/) on Real Python):

1. **Randomly choose** between constant and balanced (50/50 via [`random.choice`](https://docs.python.org/3/library/random.html#random.choice)).
2. **Build a truth table tuple** of length 16 — all same value for constant, or half `0`s and half `1`s ([shuffled](https://docs.python.org/3/library/random.html#random.shuffle)) for balanced.
3. **Wrap it in a closure** that converts four Boolean arguments to an integer index and looks up the tuple.
4. Return the closure as a callable function.

### Discussion / Interpretation

The key design choice is returning a **callable function** rather than a raw truth table array.
This is essential because Problem 2 asks for a function `determine_constant_balanced` that takes `f` as a parameter and calls it — so `f` must be callable.
Attaching `func_type` metadata to the returned function lets us verify correctness in tests without breaking the black-box contract for the actual algorithm.

Using [`random.shuffle`](https://docs.python.org/3/library/random.html#random.shuffle) to arrange the balanced outputs guarantees exactly 8 ones and 8 zeros, satisfying the balanced definition.

## Problem 1: Generating Random Boolean Functions

The [Deutsch–Jozsa algorithm](https://quantum.cloud.ibm.com/learning/en/modules/computer-science/deutsch-jozsa) works with Boolean functions that accept a fixed number of Boolean inputs and return a single Boolean output.

### Problem Statement

Write a Python function `random_constant_balanced` that returns a randomly chosen function from the set of **constant** or **balanced** functions taking **four Boolean arguments** as inputs.

### Function Types

- **Constant functions**: Always return `True` OR always return `False` (2 total)
- **Balanced functions**: Return `True` for exactly half (8) of the 16 possible input combinations

### Approach

With 4 Boolean inputs, there are 2^4 = 16 possible input combinations. I'll use a truth table approach where:
1. Generate all 16 possible input combinations using [`itertools.product`](https://docs.python.org/3/library/itertools.html#itertools.product)
2. Randomly decide whether to create a constant or balanced function
3. Build a truth table mapping inputs to outputs
4. Return a function that uses this truth table

In [ ]:
import itertools
import random

def random_constant_balanced():
    """
    Returns a randomly chosen function from the set of constant or balanced functions.
    Takes 4 Boolean arguments as inputs.
    
    Returns:
        A function that takes 4 boolean arguments and returns a boolean.
    """
    # Generate all 16 possible input combinations for 4 boolean variables
    all_inputs = list(itertools.product([False, True], repeat=4))
    
    # Randomly choose between constant or balanced function
    is_constant = random.choice([True, False])
    
    if is_constant:
        # Constant function: always return True or False
        constant_value = random.choice([True, False])
        truth_table = {inputs: constant_value for inputs in all_inputs}
    else:
        # Balanced function: return True for exactly 8 out of 16 combinations
        # Randomly select 8 combinations to return True
        true_outputs = random.sample(all_inputs, 8)
        truth_table = {inputs: (inputs in true_outputs) for inputs in all_inputs}
    
    # Return a function that uses the truth table
    def boolean_function(a, b, c, d):
        return truth_table[(a, b, c, d)]
    
    return boolean_function

In [ ]:
func = random_constant_balanced()
print(func(True, False, True, False))  # Test with sample inputs

False
